# Split mini representative dataset

In [1]:
from createCSV import TrainTest
import config

if __name__ == "__main__":
    csv = TrainTest(
        base_dir = config.BASE_DIR,
        all_csv = config.ALL_CSV,
        train_csv = config.TRAIN_CSV,
        test_csv = config.TEST_CSV,
        label_column = config.LABEL_COLUMN,
        test_size = config.TEST_SIZE,
        seed = config.SEED)
    csv.set_all_seeds()
    csv.df()

Train set length:
755
Test set length:
245
Train set label distribution:
0    398
1    357
Name: label, dtype: int64
Test set label distribution:
1    143
0    102
Name: label, dtype: int64


In [2]:
from createCSV import TrainVal
import config

if __name__ == "__main__":
    csv1 = TrainVal(
        orig_train_csv = config.TRAIN_CSV,
        train_csv = config.TRAIN_CSV_1,
        val_csv = config.VAL_CSV_1,
        label_column = config.LABEL_COLUMN,
        val_size = config.VAL_SIZE,
        seed = config.SEED)
    csv1.set_all_seeds()
    csv1.df()

Original train set length:
755
New train set length and first n rows:
613
                                           file_name  label   parcel_id
8   22-0249080_F796528C401EECCDE0530EEE260A12DE.jpeg      0  22-0249080
9   22-0249080_F796528C45EEECCDE0530EEE260A12DE.jpeg      0  22-0249080
16  22-0249181_F7F34037674E9508E0530EEE260A6CC9.jpeg      0  22-0249181
17  22-0249181_F7F3403767749508E0530EEE260A6CC9.jpeg      0  22-0249181
18  22-0249181_F7F3403767E39508E0530EEE260A6CC9.jpeg      0  22-0249181
Val set length and first n rows:
142
                                          file_name  label   parcel_id
0  22-0248171_F796528C3E03ECCDE0530EEE260A12DE.jpeg      0  22-0248171
1  22-0248171_F796528C3E05ECCDE0530EEE260A12DE.jpeg      0  22-0248171
2  22-0248171_F796528C4450ECCDE0530EEE260A12DE.jpeg      0  22-0248171
3  22-0248171_F796528C4451ECCDE0530EEE260A12DE.jpeg      0  22-0248171
4  22-0248171_F796528C4453ECCDE0530EEE260A12DE.jpeg      0  22-0248171
Train set label distribution:
1

# Train model on mini representative dataset

In [4]:
# Train model
from model import NN
from customDataModule import CustomDataModule
import config
import pytorch_lightning as pl
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch import seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint
import torch
import torchvision.models as models
import torch.nn as nn

seed_everything(
    42, workers=True
)  # By setting workers=True in seed_everything(), Lightning derives unique seeds across all dataloader workers and processes for torch, numpy and stdlib random number generators. When turned on, it ensures that e.g. data augmentations are not repeated across workers.



if __name__ == "__main__":
    
    model = models.vgg16(weights='VGG16_Weights.DEFAULT')
    
    logger = TensorBoardLogger("tb_logs", name="view") # tb_logs is the folder, name is the name of the experiment/model
    model = NN(
        model=model,
        input_shape=config.INPUT_SHAPE,
        num_classes=config.NUM_CLASSES,
        learning_rate=config.LEARNING_RATE,
    )  # .to(device)
    dm = CustomDataModule(
        data_dir=config.DATA_DIR,
        train_csv=config.TRAIN_CSV_1,
        val_csv=config.VAL_CSV_1,
        test_csv=config.TEST_CSV,
        batch_size=config.BATCH_SIZE,
        num_workers=config.NUM_WORKERS,
    )
    

    
    trainer = pl.Trainer(
        logger=logger, # PyTorch lightning will automatically know what we are logging by looking at our model.py logs
        accelerator=config.ACCELERATOR,
        devices=config.DEVICES,
        min_epochs=config.MIN_EPOCHS,
        max_epochs=config.MAX_EPOCHS,
        deterministic=config.DETERMINISTIC,
#        callbacks=[ModelCheckpoint(monitor="val_loss", mode="min")],
    )
      # deterministic ensures random seed reproducibility

    trainer.fit(model, dm)  # it will automatically know which dataloader to use

# A general place to start is to set num_workers equal to the number of CPU cores on that machine. You can get the number of CPU cores in python using os.cpu_count(), but note that depending on your batch size, you may overflow RAM memory.


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                | Params
--------------------------------------------------------
0 | model           | VGG                 | 138 M 
1 | classifier      | Linear              | 2.0 K 
2 | loss_fn         | CrossEntropyLoss    | 0     
3 | train_acc       | MulticlassAccuracy  | 0     
4 | val_acc         | MulticlassAccuracy  | 0     
5 | train_recall    | MulticlassRecall    | 0     
6 | train_precision | MulticlassPrecision | 0     
7 | train_f1score   | MulticlassF1Score   | 0     
--------------------------------------------------------
2.0 K     Trainable params
138 M     Non-trainable params
138 M     Total params
553.438   Total estimated model params size (MB)


C:\Users\kaspe\.conda\envs\LBST\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (20) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 20/20 [00:10<00:00,  1.86it/s, v_num=1]
Validation: 0it [00:00, ?it/s]
Epoch 1: 100%|██████████| 20/20 [00:10<00:00,  1.93it/s, v_num=1, train_loss=0.827, train_accuracy=0.584, train_recall=0.584, train_precision=0.584, train_f1score=0.584]
Validation: 0it [00:00, ?it/s]
Epoch 2: 100%|██████████| 20/20 [00:10<00:00,  1.98it/s, v_num=1, train_loss=0.573, train_accuracy=0.713, train_recall=0.713, train_precision=0.713, train_f1score=0.713]
Validation: 0it [00:00, ?it/s]
Epoch 3: 100%|██████████| 20/20 [00:10<00:00,  1.94it/s, v_num=1, train_loss=0.605, train_accuracy=0.690, train_recall=0.690, train_precision=0.690, train_f1score=0.690]
Validation: 0it [00:00, ?it/s]
Epoch 4: 100%|██████████| 20/20 [00:10<00:00,  1.97it/s, v_num=1, train_loss=0.566, train_accuracy=0.718, train_recall=0.718, train_precision=0.718, train_f1score=0.718]
Validation: 0it [00:00, ?it/s]
Epoch 5: 100%|██████████| 20/20 [00:10<00:00,  1.91it/s, v_num=1, train_loss=0.607, train_accuracy=

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 20/20 [00:18<00:00,  1.11it/s, v_num=1, train_loss=0.532, train_accuracy=0.773, train_recall=0.773, train_precision=0.773, train_f1score=0.773]


In [7]:
trainer.validate(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation DataLoader 0: 100%|██████████| 5/5 [00:01<00:00,  4.65it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       val_accuracy        │    0.7394366264343262     │
│         val_loss          │    0.6064642071723938     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.6064642071723938, 'val_accuracy': 0.7394366264343262}]

In [8]:
trainer.test(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 8/8 [00:01<00:00,  4.35it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.5881527066230774     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 0.5881527066230774}]

In [9]:
model

NN(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16): MaxPo